In [1]:
# loading all the dataset for biometric
import pandas as pd

bio1 = pd.read_csv(r'C:\Users\ankit\Desktop\uidai\data_set\api_data_aadhar_biometric\api_data_aadhar_biometric_0_500000.csv')
bio2 = pd.read_csv(r'C:\Users\ankit\Desktop\uidai\data_set\api_data_aadhar_biometric\api_data_aadhar_biometric_500000_1000000.csv')
bio3 = pd.read_csv(r'C:\Users\ankit\Desktop\uidai\data_set\api_data_aadhar_biometric\api_data_aadhar_biometric_1000000_1500000.csv')
bio4 = pd.read_csv(r'C:\Users\ankit\Desktop\uidai\data_set\api_data_aadhar_biometric\api_data_aadhar_biometric_1500000_1861108.csv')

biometric_data = pd.concat([bio1, bio2, bio3, bio4], ignore_index=True)

In [3]:
biometric_data.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,Haryana,Mahendragarh,123029,280,577
1,01-03-2025,Bihar,Madhepura,852121,144,369
2,01-03-2025,Jammu and Kashmir,Punch,185101,643,1091
3,01-03-2025,Bihar,Bhojpur,802158,256,980
4,01-03-2025,Tamil Nadu,Madurai,625514,271,815


In [13]:
# Gap Identification 

'''
Update Rate--> expected Population (5-17), Based on threshold we can identify the gap
Pincode clustring --> lat,long
'''


'\nUpdate Rate--> expected Population (5-17), Based on threshold we can identify the gap\nPincode clustring --> lat,long\n'

In [ ]:
# These are the disctricts where child biometric updates are significantly lower than adult updates and needed urgent attention for MBU compliance campaigns.

# 1. Calculate the ratio of child to adult updates as a proxy for MBU compliance
biometric_data['mbu_compliance_ratio'] = biometric_data['bio_age_5_17'] / ( biometric_data['bio_age_17_'] + 1)  # Adding 1 to avoid division by zero

# 2. Identify outliers (Districts with significantly low child activity)
mean_ratio = biometric_data['mbu_compliance_ratio'].mean()
gap_districts = biometric_data[biometric_data['mbu_compliance_ratio'] < (mean_ratio * 0.5)]

print("Districts requiring urgent awareness campaigns:")
print(gap_districts[['state', 'district', 'mbu_compliance_ratio']])
len(set(gap_districts['district'])) # 970 districts

Districts requiring urgent awareness campaigns:
               state                    district  mbu_compliance_ratio
0            Haryana                Mahendragarh              0.484429
1              Bihar                   Madhepura              0.389189
3              Bihar                     Bhojpur              0.260958
4         Tamil Nadu                     Madurai              0.332108
5        Maharashtra                   Ratnagiri              0.292453
...              ...                         ...                   ...
1861102  West Bengal  South Twenty Four Parganas              0.111111
1861103  West Bengal              Uttar Dinajpur              0.400000
1861104  West Bengal              Uttar Dinajpur              0.000000
1861105  West Bengal              West Midnapore              0.000000
1861107  West Bengal              West Midnapore              0.000000

[779069 rows x 3 columns]


970

In [ ]:
# use these spikes to predict future demand and plan resource allocation accordingly

gap_districts = biometric_data[biometric_data['mbu_compliance_ratio'] > (mean_ratio * 5)]
print("Districts with good MBU compliance:")
print(gap_districts[['state', 'district', 'mbu_compliance_ratio']])
#--- IGNORE ---
len(set(gap_districts['district'])) # 723 districts

Districts with good MBU compliance:
                  state           district  mbu_compliance_ratio
122      Andhra Pradesh         Srikakulam              6.854545
194             Gujarat    Devbhumi Dwarka              9.000000
198          Tamil Nadu         Tirupattur             11.750000
275               Assam            Barpeta             13.142857
391      Andhra Pradesh      Ananthapuramu              7.038462
...                 ...                ...                   ...
1860517  Andhra Pradesh      East Godavari              7.000000
1860615  Andhra Pradesh            Kurnool              9.000000
1860736     Uttarakhand           Dehradun              7.500000
1860800     West Bengal         Barddhaman             10.000000
1860994     West Bengal  North 24 Parganas              6.000000

[35049 rows x 3 columns]


723

In [ ]:
# district with almost zero biometric updates needs technical intervention 

biometric_data['total_biometric_updates'] = biometric_data['bio_age_5_17'] + biometric_data['bio_age_17_']
quarter = biometric_data['total_biometric_updates'].describe().reset_index().iloc[4].values[1]  # Getting the 25th percentile value

zero_update_districts = biometric_data[biometric_data['total_biometric_updates'] < (quarter * 0.1 )]  # Threshold set at 10% of the 25th percentile
print("Districts with very low biometric updates:")
print(zero_update_districts[['state', 'district', 'total_biometric_updates']])
len(set(zero_update_districts['district'])) # 10 districts


Districts with very low biometric updates:
                 state    district  total_biometric_updates
179473       Karnataka    Belagavi                        0
180011       Karnataka    Belagavi                        0
204209  Andhra Pradesh    Tirupati                        0
228413       Telangana     Jangoan                        0
304353       Telangana    Jagitial                        0
328393           Assam      Jorhat                        0
387403   Uttar Pradesh      Ballia                        0
403404  Andhra Pradesh    Kakinada                        0
427084       Telangana    Siddipet                        0
448396      Tamil Nadu  Villupuram                        0
456514       Telangana    Jagitial                        0
483890   Uttar Pradesh     Ayodhya                        0


10

In [ ]:
# predictive model